In [157]:
import random
import numpy as np
import os
from os.path import join
import torch
from multiprocessing.dummy import Pool
from tqdm import tqdm
import cv2

In [158]:
import matplotlib.pyplot as plt

In [159]:
scale = 2
input_dir = f"HR_raw"
output_dir = f"LR_rawx{scale}" 
test = False
mode="EXTREME"

Function

In [160]:
def show(img):
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [161]:
def crop_ref(image, scale,offset=(0,0)):
	heightHR, widthHR = image.shape[:2]
	heightRef, widthRef = heightHR // scale , widthHR // scale 
	downRef, rightRef = offset
	crop_coord = [downRef, downRef+heightRef, rightRef, rightRef+widthRef]
	return image[downRef:downRef+heightRef, rightRef:rightRef+widthRef], crop_coord


In [162]:
def crop_multi_ref(image, scale, mode="REGULAR"):
	heightHR, widthHR = image.shape[:2]
	heightRef, widthRef = heightHR // scale , widthHR // scale 
	offset = []
	if mode=="REGULAR":
		offsets = [
			((heightHR-heightRef) // 2,(widthHR-widthRef) // 2),
			((heightHR-heightRef) // 4,(widthHR-widthRef) // 4),
			((heightHR-heightRef) // 4,(widthHR-widthRef)*3 // 4),
			((heightHR-heightRef)*3 // 4,(widthHR-widthRef) // 4),
			((heightHR-heightRef)*3 // 4,(widthHR-widthRef)*3 // 4),
		]
	elif mode=="EXTREME":
		offsets = [
			((heightHR-heightRef) // 2,(widthHR-widthRef) // 2),
			(0,0),
			(0,widthHR-widthRef),
			(heightHR-heightRef,0),
			(heightHR-heightRef,widthHR-widthRef),
		]
	output = [crop_ref(image, scale,offset) for offset in offsets]
	return output

Main

In [163]:
for input in os.listdir(input_dir):  
    name, ext = os.path.splitext(input)
    input_full = os.path.join(input_dir, name+ext )
    img = cv2.imread(input_full)
    assert img is not None

    output = crop_multi_ref(img,scale,mode)

    os.makedirs(output_dir, exist_ok = True) 
    for idx, i in enumerate(output):
        i, _ = i
        output_full = os.path.join(output_dir, f"{name}-ref{idx}{ext}")
        assert cv2.imwrite(output_full, i)